In [103]:
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

from keras.layers import Dense ,LSTM,concatenate,Input,Flatten
from keras.models import Model
from keras.callbacks import EarlyStopping

import openpyxl

In [104]:
dataset = pd.read_excel('tres.xlsx')  
dataset.tail()

,EFICIENCIATERMINAL,SOSTENIMIENTO_ESTATAL,SOSTENIMIENTO_FEDERAL,SOSTENIMIENTO_FEDERALIZADO,SOSTENIMIENTO_PARTICULAR,DESERCIÓNINTRACURRICULAR,REPROBACIÓN,REPROBACIÓNCONREGULARIZADOS,Porcentajedepoblaciónde15añosomásanalfabeta,Porcentajedepoblaciónde15añosomássinprimariacompleta,...,PROGRAMA_PROFESIONALMEDIO,PROGRAMA_TECNICA,PROGRAMA_TECNOLOGICO,PROGRAMA_TELESECUNDARIA,NOMTURNO_CONTINUO(JORNADAAMPLIADA),NOMTURNO_CONTINUO(TIEMPOCOMPLETO),NOMTURNO_DISCONTINUO,NOMTURNO_MATUTINO,NOMTURNO_NOCTURNO,NOMTURNO_VESPERTINO
6252,0.8650,0,1,0,0,0.0234,0.1627,0.1455,0.0608,0.2494,...,0,0,1,0,0,0,0,1,0,0
6253,0.7500,1,0,0,0,0.1364,0.1579,0.0643,0.0608,0.2494,...,0,0,0,0,0,0,0,1,0,0
6254,0.0000,1,0,0,0,0.2520,0.3263,0.0737,0.0608,0.2494,...,0,0,0,0,0,0,0,0,0,1
6255,0.7353,1,0,0,0,0.0115,0.0116,0.0000,0.0608,0.2494,...,0,0,0,0,0,0,0,0,0,1
6256,1.0000,0,0,0,1,1.0000,0.0000,0.0000,0.0608,0.2494,...,0,0,0,0,0,0,0,0,0,1


In [105]:
train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)

In [106]:
train_stats = train_dataset.describe()
train_stats.pop("EFICIENCIATERMINAL")
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
SOSTENIMIENTO_ESTATAL,5006.0,0.227127,0.419017,0.0000,0.0000,0.0000,0.000000,1.0000
SOSTENIMIENTO_FEDERAL,5006.0,0.083700,0.276964,0.0000,0.0000,0.0000,0.000000,1.0000
SOSTENIMIENTO_FEDERALIZADO,5006.0,0.503995,0.500034,0.0000,0.0000,1.0000,1.000000,1.0000
SOSTENIMIENTO_PARTICULAR,5006.0,0.185178,0.388481,0.0000,0.0000,0.0000,0.000000,1.0000
DESERCIÓNINTRACURRICULAR,5006.0,0.040310,0.085792,0.0000,0.0000,0.0122,0.045975,1.0000
REPROBACIÓN,5006.0,0.030997,0.081413,0.0000,0.0000,0.0000,0.021575,1.0000
REPROBACIÓNCONREGULARIZADOS,5006.0,0.013593,0.058243,0.0000,0.0000,0.0000,0.000000,1.0000
Porcentajedepoblaciónde15añosomásanalfabeta,5006.0,0.048490,0.038499,0.0173,0.0205,0.0389,0.061900,0.2533
Porcentajedepoblaciónde15añosomássinprimariacompleta,5006.0,0.185176,0.086887,0.0925,0.0934,0.1721,0.249400,0.4762
Porcentajedeocupantesenviviendassindrenajeniexcusado,5006.0,0.023768,0.079546,0.0001,0.0005,0.0045,0.015100,0.5449


In [107]:
train_labels = train_dataset.pop("EFICIENCIATERMINAL")
test_labels = test_dataset.pop("EFICIENCIATERMINAL")

In [108]:
def norm(x):
      return (x - train_stats['mean']) / train_stats['std']
#normed_train_data = norm(train_dataset)
#normed_test_data = norm(test_dataset)
normed_train_data = train_dataset #norm(train_dataset)
normed_test_data = test_dataset #norm(test_dataset)
#normed_train_data

In [109]:
def build_model():
    model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
    ])

    optimizer = tf.keras.optimizers.RMSprop(0.001)

    model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
#    model.compile(optimizer='rmsprop',
#              loss='binary_crossentropy',
#              metrics=['accuracy'])
    return model



In [110]:
model = build_model()

In [111]:
model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_58 (Dense)             (None, 64)                2688      
_________________________________________________________________
dense_59 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_60 (Dense)             (None, 1)                 65        
Total params: 6,913
Trainable params: 6,913
Non-trainable params: 0
_________________________________________________________________


In [112]:
example_batch = normed_train_data[:10]
example_result = model.predict(example_batch)
example_result

array([[0.566],
       [0.615],
       [0.409],
       [0.32 ],
       [0.425],
       [0.586],
       [0.506],
       [0.397],
       [0.642],
       [0.441]], dtype=float32)

In [ ]:
# Display training progress by printing a single dot for each completed epoch
class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0: print('')
        print('.', end='')

EPOCHS = 1000

history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[PrintDot()])


....................................................................................................
....................................................................................................
......

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error [EFICIENCIA TERMINAL]')
  plt.plot(hist['epoch'], hist['mae'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mae'],
           label = 'Val Error')
  plt.ylim([0,5])
  plt.legend()

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Square Error [$EFICIENCIA_TERMINAL^2$]')
  plt.plot(hist['epoch'], hist['mse'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mse'],
           label = 'Val Error')
  plt.ylim([0,20])
  plt.legend()
  plt.show()


plot_history(history)

In [ ]:
model = build_model()

# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(normed_train_data, train_labels, epochs=EPOCHS,
                    validation_split = 0.2, verbose=0, callbacks=[early_stop, PrintDot()])

plot_history(history)

In [ ]:
loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} EFICIENCIATERMINAL".format(mae))

In [ ]:
test_predictions = model.predict(normed_test_data).flatten()

plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [EFICIENCIATERMINAL]')
plt.ylabel('Predictions [EFICIENCIATERMINAL]')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
_ = plt.plot([-100, 100], [-100, 100])

In [ ]:
error = test_predictions - test_labels
plt.hist(error, bins = 25)
plt.xlabel("Prediction Error [EFICIENCIATERMINAL]")
_ = plt.ylabel("Count")